In [9]:
import pandas as pd 
df = pd.read_csv('fra.txt', delimiter='\t', names=['english', 'french', 'attribute'], usecols=['english', 'french'])
df.reset_index(drop=True)
df.head()

,english,french
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !


In [11]:
en = df['english']
fr = df['french']

en_max_len = max(len(line.split()) for line in en)
fr_max_len = max(len(line.split()) for line in fr)
sequence_len = max(en_max_len, fr_max_len)

print(f'Max phrase length (English): {en_max_len}')
print(f'Max phrase length (French): {fr_max_len}')
print(f'Sequence length: {sequence_len}')

Max phrase length (English): 55
Max phrase length (French): 68
Sequence length: 68


In [71]:
from tokenizers import SentencePieceBPETokenizer
from transformers import PreTrainedTokenizerFast
from torch.nn.utils.rnn import pad_sequence
import torch
from tqdm import tqdm

english_tokenizer = SentencePieceBPETokenizer()
english_tokenizer.train_from_iterator(
    df['english'],
    vocab_size=30_000,
    min_frequency=2,
    show_progress=True,
    limit_alphabet=500,
)
english_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=english_tokenizer
)
eng_eng_encoded = []
for i in tqdm(range(len(df['english']))):
    eng_eng_encoded.append(torch.tensor(english_tokenizer.encode(df['english'].iloc[i]), dtype=torch.long))
english_padded_encoded = pad_sequence(eng_eng_encoded, batch_first = True)


100%|██████████| 232736/232736 [00:11<00:00, 20188.46it/s]


In [72]:
french_tokenizer = SentencePieceBPETokenizer()
french_tokenizer.train_from_iterator(
    df['french'],
    vocab_size=30_000,
    min_frequency=5,
    show_progress=True,
    limit_alphabet=500,
)
french_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=french_tokenizer
)
french_eng_encoded = []
for i in tqdm(range(len(df['french']))):
    french_eng_encoded.append(torch.tensor(french_tokenizer.encode(df['french'].iloc[i]), dtype=torch.long))
french_padded_encoded = pad_sequence(french_eng_encoded, batch_first = True)

100%|██████████| 232736/232736 [00:12<00:00, 19005.42it/s]


In [76]:
eng_eng_encoded[-1]

tensor([   93,   626,  3433,   219,   351,   152,   207, 22066,  9771,   274,
          815,  1383, 22394,   235,   309,  4640,   129,   190,   548,    92,
         1350,   529,    92, 25223,  5621, 18296,  3336,   503,   370,   361,
           93,   200,  3433,   219,   271,  9654,  3880,     2, 11370,   464,
        11922,  6395,   103,  1044,   136,   510,   785,   376,  7973,   274,
           93,   376,   191,   102,   200,  3433,   219,   279,   691,     2])

In [77]:
t = eng_eng_encoded + (french_eng_encoded)
len(t)

465472

In [90]:
a = pad_sequence(t, batch_first=True)
X, y = a[:len(eng_eng_encoded)], a[len(eng_eng_encoded):]

In [95]:
X.size()

torch.Size([232736, 73])

In [93]:
len(english_tokenizer.get_vocab())

29760

In [94]:
len(french_tokenizer.get_vocab())

24053

In [97]:
y[:10][:, :-1].size()

torch.Size([10, 72])

In [99]:
from transformer import Transformer
model = transformer = Transformer(29760, 24053, 512, 8, 6, 2048, 73, 0.1)
model.train()
g, h = X[:10], y[:10]
output = transformer(g, h[:, :-1])

AttributeError: 'MultiHeadAttention' object has no attribute 'd_k'

In [120]:
import torch.nn as nn
from transformer import * 
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)
transformer.train()
# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
criterion = nn.CrossEntropyLoss(ignore_index=0)

output = transformer(src_data, tgt_data[:, :-1])

AttributeError: 'MultiHeadAttention' object has no attribute 'd_k'